In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

def fill_gap(frame):

    h, w = frame.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    im_floodfill = frame.copy()
 
    cv2.floodFill(im_floodfill, mask, (0,0), 255);
 
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
 
    # Combine the two images to get the foreground.
    im_out = frame | im_floodfill_inv
    return im_out

def get_edge(frame):
    kerneld = np.ones((5,5),np.uint8)
    kernele = np.ones((3,3),np.uint8)
    edges = cv2.Canny(frame,100,100)
    edges = cv2.dilate(edges,kerneld,iterations = 1) #5
    edges = cv2.erode(edges,kernele,iterations = 1) #5
    #edges = fill_gap(edges)
    #h, w = edges.shape
    #cv2.line(edges, (0,h-1), (w-1,h-1), (255, 255, 255), 2)
    return edges

def get_thresh(frame):
    #res, thresh = cv2.threshold(frame,70,255,cv2.THRESH_TOZERO)
    res, thresh = cv2.threshold(frame,100,255,cv2.THRESH_BINARY)
    thresh = cv2.medianBlur(thresh,5)
    return thresh

def get_center_of_mass(contours):
    c = max(contours, key=cv2.contourArea)
    M = cv2.moments(c)
 
    area = M['m00']
    if (area!=0):
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    return cx,cy

cap = cv2.VideoCapture('video/seguidor_red.mp4')
font = cv2.FONT_HERSHEY_SIMPLEX
hist = 30
while(cap.isOpened()):
    ret, frame = cap.read()

    h,w,c = frame.shape
    #print (h)
    #print (w)
    frame_c = frame[int(h/2):h,int(w/3):w]
    gray = cv2.cvtColor(frame_c, cv2.COLOR_BGR2GRAY)
    h,w = gray.shape
    thresh = get_thresh(gray)
    #thresh = get_edge(gray)
    contours,hierarchy = cv2.findContours(thresh.copy(), 1, cv2.CHAIN_APPROX_NONE)
    if (len(contours)>0):
        cx, cy = get_center_of_mass(contours)
        
        cv2.line(frame,(int(w/2)+cx,0),(int(w/2)+cx,2*h),(255,255,255),1)
        cv2.line(frame,(0,cy+int(h/3)),(w*3,cy+int(h/3)),(255,255,255),1)
 
        cv2.drawContours(frame, contours, -1, (0,0,255), 2, offset=(int(w/2),int(h)))
 
        if cx >= (w/2+hist):
           cv2.putText(frame,'Turn Right',(10,30), font, 1,(255,255,255),2)
 
        if cx < (w/2+hist) and cx > (w/2-hist):
           cv2.putText(frame,'On Track',(int(w/2-10),int(h/2)), font, 1,(255,255,255),2)
            
        if cx <= (w/2-hist):
           cv2.putText(frame,'Turn Left',(w-40,30), font, 1,(255,255,255),2)
 
    else:
        cv2.putText(frame,'y la linea?',(int(w/2),int(h/2)), font, 1,(255,255,255),2)
 
    #Display the resulting frame
    cv2.imshow('frame',frame)
    cv2.imshow('thresh',thresh)
    
    cv2.imshow('gray',gray)
    if cv2.waitKey(100) & 0xFF == ord('q'):
        #Press 'q' to close
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'